In [47]:
# Library to process MIDI files
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense, Activation, Dropout, Flatten
from tensorflow.keras.layers import Conv1D,MaxPooling1D, Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Embedding

from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

In [2]:
import mido
from mido import MidiFile, MidiTrack, Message

In [3]:
music_list = pd.read_csv('music_list.csv')
music_list = music_list.fillna('Not Available')
music_list

,1-Calming,2-Sports,3-Mysterious,4-Nostalgia,5-Happy
0,Big Sky,Shiatsu,Vague,All That You Will Be,Shadows in Vienna
1,Maybe Another Day,Kermode,Cloak,Fractured Memories,Magic Forest
2,The Human Derivative,Run Till Dawn,Lethal Secrets,When It Rains It Pours,Mistaken by Accident
3,The Crossing,Whatever Man,Remote,Near Her,New Orleans Diner
4,A Presence Felt,Body Work,The Mole,Days,Life in Tinseltown
5,No Good Byes,Space Palace,The Unveiling,Mornings,Harlem Rollout
6,Awash,Black Coffee,Ginger Root,Always the Same,Summertime Memory
7,A Whisper In The Dark,Unleash My Halo,Fiona,Through These Doors,Countryside Living
8,Whisper of the Pines,Way Way Back,Dust Blaster,Heading Home Again,On the Rooftop
9,Traces,The Sooner The Better,Fighter Jet,A Lighthouse In Space,The Upper Hand


In [4]:
# find a filename that contains a string

# import os

# folder_name = '1-Calming'
# keyword = 'A Presence Felt'
# for fname in os.listdir('midi/'+folder_name):
#     if keyword in fname:
#         print(fname, "has the keyword")

In [5]:
# prepare a dictionary for song names for each category

import os
song_dict = {}
for folder_name in music_list.columns:
#     print(folder_name, '\n')
    song_dict[folder_name] = []
    for fname in os.listdir('midi/'+folder_name):
#         print('fname: ', fname)
        song_dict[folder_name].append(fname)

print(song_dict['1-Calming'])

['ES_Big Sky - Gavin Luke.mp3.mid', 'ES_The Crossing - Gavin Luke.mp3.mid', '.DS_Store', 'ES_Maybe Another Day - Spectacles Wallet and Watch.mp3.mid', 'ES_No Good Byes - Amaranth Cove.mp3.mid', 'ES_A Presence Felt - Gavin Luke.mp3.mid', 'ES_Awash - Gavin Luke.mp3.mid', 'ES_A Whisper In The Dark - Gavin Luke.mp3.mid', 'ES_Onthou - Ever So Blue.mp3.mid', 'ES_The Human Derivative - Amaranth Cove.mp3.mid', 'ES_Happy Days Are Long Awaited - River Foxcroft.mp3.mid']


In [6]:
song_dict.keys()

dict_keys(['1-Calming', '2-Sports', '3-Mysterious', '4-Nostalgia', '5-Happy'])

In [7]:
# X = []
# y = []
# for cat in song_dict.keys():
#     for fname in song_dict[cat]:
#         if fname != '.DS_Store':
#             y.append(cat)
#     #         midi_filename = 'midi/1-calming/ES_A Presence Felt - Gavin Luke.mp3.mid'
#             midi_filename = 'midi/' + cat + '/' + fname
# #             print(midi_filename)
#             midi = MidiFile(midi_filename)
#             cnt = 0
#             for msg in midi:
#                 print(msg)
#                 cnt+=1
#                 if cnt>100:
#                     break 

#             # save notes for the song as a list
#             notes = []
#             for msg in midi:
#                 if not msg.is_meta and msg.channel == 0 and msg.type == 'note_on':
#                     data = msg.bytes()
#                     notes.append(data[1]) # we are just adding the note!

#             ## rescale all the notes to the [0,1] interval
#             scaler = MinMaxScaler(feature_range=(0,1))
#             scaler.fit(np.array(notes).reshape(-1,1))
#             notes = list(scaler.transform(np.array(notes).reshape(-1,1)))
#             notes = [list(note)[0] for note in notes]

#             X.append(notes)
        
# print(X)
# print(y)


In [8]:
# df_X = pd.DataFrame(X)
# df_X.to_csv('df_X.csv', index=False)
# df_y = pd.DataFrame(y)
# df_y.to_csv('df_y.csv', index=False)

In [9]:
df_X = pd.read_csv('df_X.csv')
df_X = df_X.fillna(0)
df_X

,0,1,2,3,4,5,6,7,8,9,...,44831,44832,44833,44834,44835,44836,44837,44838,44839,44840
0,0.250000,0.375000,0.535714,0.607143,0.482143,0.607143,0.267857,0.464286,0.607143,0.660714,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.662162,0.297297,0.500000,0.297297,0.391892,0.554054,0.297297,0.554054,0.297297,0.297297,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.462500,0.550000,0.662500,0.762500,0.787500,0.850000,0.300000,0.312500,0.525000,0.787500,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.641791,0.194030,0.417910,0.208955,0.223881,0.238806,0.223881,0.582090,0.358209,0.701493,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.610169,0.288136,0.542373,0.338983,0.288136,0.288136,0.288136,0.338983,0.406780,0.288136,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.448718,0.294872,0.602564,0.448718,0.692308,0.692308,0.384615,0.692308,0.500000,0.692308,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.455882,0.529412,0.250000,0.264706,0.544118,0.617647,0.676471,0.500000,0.544118,0.279412,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.392857,0.607143,0.089286,0.089286,0.089286,0.517857,0.089286,0.464286,0.607143,0.303571,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.512500,0.600000,0.350000,0.362500,0.450000,0.362500,0.362500,0.362500,0.400000,0.537500,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.355263,0.236842,0.342105,0.552632,0.355263,0.802632,0.236842,0.342105,0.552632,0.381579,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [10]:
X_read = np.array([df_X.iloc[i,:].tolist() for i in range(df_X.shape[0])])

In [11]:
X_read.shape

(50, 44841)

In [12]:
df_y = pd.read_csv('df_y.csv')
df_y.shape

(50, 1)

In [13]:
y = df_y['0'].tolist()

In [14]:
# generate dummy variable y

from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

# encode class values as integers
encoder = LabelEncoder()
encoder.fit(y)
encoded_Y = encoder.transform(y)
encoded_Y
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)
dummy_y

array([[1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0.

In [112]:
# split datasets into train and test
X_train, X_test, y_train, y_test = train_test_split(X_read, dummy_y, test_size = 0.3, 
                                                    random_state = 0)

In [113]:
# set the length of all sequences to be equal to max_len

max_len = 0
temp_len = 0
for i in range(len(X_read)):
    max_len = len(X_read[i])
    if max_len < temp_len:
        max_len = temp_len
    else:
        temp_len = max_len
#     print(max_len)
max_len

# X_train = sequence.pad_sequences(X_train, maxlen=max_len)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1],1))

# X_test = sequence.pad_sequences(X_test, maxlen=max_len)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)


In [115]:
# X_train

In [116]:
y_train.shape

(35, 5)

In [117]:
max_len

44841

In [126]:
# Define a model
n_prev = 1000
embedding_vector_length = 300
# max_len = 1000

model = Sequential()
# model.add(Embedding(n_prev, embedding_vector_length, input_length=max_len))

model.add(Conv1D(filters=32, kernel_size=8, strides=1, activation='relu', 
                 input_shape=(n_prev, 1)))
model.add(MaxPooling1D(pool_size=4))

model.add(LSTM(256 , input_shape=(n_prev, 1), return_sequences=True))
# model.add(Dropout(0.3))


model.add(Conv1D(filters=32, kernel_size=8, strides=1, activation='relu', 
                 input_shape=(n_prev, 1)))
model.add(MaxPooling1D(pool_size=4))

model.add(LSTM(128, input_shape=(n_prev, 1), return_sequences=False))
# model.add(Dropout(0.3))

# model.add(LSTM(64, input_shape=(n_prev, 1), return_sequences=False))
# model.add(Dropout(0.3))

model.add(Dense(128, activation = 'relu'))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(5, activation = 'sigmoid'))

optimizer = Adam(lr=0.001)
model.compile(loss='categorical_crossentropy', 
              optimizer=optimizer, metrics=['accuracy'])
print(model.summary())

# train model
model.fit(np.array(X_train[:,:n_prev]), np.array(y_train), 
          32, 20, verbose=1)#, callbacks=[model_save_callback])

Model: "sequential_25"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_12 (Conv1D)           (None, 993, 32)           288       
_________________________________________________________________
max_pooling1d_9 (MaxPooling1 (None, 248, 32)           0         
_________________________________________________________________
lstm_23 (LSTM)               (None, 248, 256)          295936    
_________________________________________________________________
conv1d_13 (Conv1D)           (None, 241, 32)           65568     
_________________________________________________________________
max_pooling1d_10 (MaxPooling (None, 60, 32)            0         
_________________________________________________________________
lstm_24 (LSTM)               (None, 128)               82432     
_________________________________________________________________
dense_44 (Dense)             (None, 128)             

In [125]:
X_test_trunc = X_test[:, :n_prev]
scores = model.evaluate(X_test_trunc.reshape(X_test_trunc.shape[0], X_test_trunc.shape[1], 1),
                        y_test, verbose=0)
print("Out-of-Sample Accuracy: %.2f%%" % (scores[1]*100))


Out-of-Sample Accuracy: 13.33%
